In [1]:
import pandas as pd
import numpy as np

Import data as a dataframe

In [2]:
df = pd.read_csv("data/Week1_challenge_data_source.csv")

Before Cleaning the dataset, we explore what has been stored in it. The column names, how many data points, number of colum etc. All these questions needs to be answered

In [5]:
df.shape

(150001, 55)

The data has 150001 rows and 55 columns

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

How much data is missing?

In [10]:
df.shape[0]

150001

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [41]:
def percent_missing(df: pd.DataFrame, cols=None):
    full_num_enteries = df.shape[0]
    
    if not cols:
        cols = df.columns.to_list()
        
    for col in cols:
        num_missing = df[col].isnull().sum()
        perc_missing = num_missing/full_num_enteries*100
        
        print("{}  missing values are ------------- {:.2f}% ".format(col, perc_missing))

In [40]:
percent_missing(df)

Bearer Id  missing values are ------------- 0.66% 
Start  missing values are ------------- 0.00% 
Start ms  missing values are ------------- 0.00% 
End  missing values are ------------- 0.00% 
End ms  missing values are ------------- 0.00% 
Dur. (ms)  missing values are ------------- 0.00% 
IMSI  missing values are ------------- 0.38% 
MSISDN/Number  missing values are ------------- 0.71% 
IMEI  missing values are ------------- 0.38% 
Last Location Name  missing values are ------------- 0.77% 
Avg RTT DL (ms)  missing values are ------------- 18.55% 
Avg RTT UL (ms)  missing values are ------------- 18.54% 
Avg Bearer TP DL (kbps)  missing values are ------------- 0.00% 
Avg Bearer TP UL (kbps)  missing values are ------------- 0.00% 
TCP DL Retrans. Vol (Bytes)  missing values are ------------- 58.76% 
TCP UL Retrans. Vol (Bytes)  missing values are ------------- 64.43% 
DL TP < 50 Kbps (%)  missing values are ------------- 0.50% 
50 Kbps < DL TP < 250 Kbps (%)  missing values are ---

We see that some columns are missing a lot of values. Handeling 